In [ ]:
# Jinseok Ryu, Ph.D.
# jinseuk56@gmail.com
# 20230927

import py4DSTEM
import tkinter.filedialog as tkf
import numpy as np
import tifffile
import matplotlib.pyplot as plt
py4DSTEM.__version__


def fourd_roll_axis(stack):
    stack = np.rollaxis(np.rollaxis(stack, 2, 0), 3, 1)
    return stack

def spike_remove(data, percent_thresh, mode):

    pacbed = np.mean(data, axis=(0, 1))
    intensity_integration_map = np.sum(data, axis=(2, 3))

    threshold = np.percentile(intensity_integration_map, percent_thresh)
    if mode == "upper":
        spike_ind = np.where(intensity_integration_map > threshold)
    elif mode == "lower":
        spike_ind = np.where(intensity_integration_map < threshold)
    else:
        print("Wrong mode!")
        return

    print("threshold value = %f"%threshold)
    print("number of abnormal pixels = %d"%len(spike_ind[0]))

    data[spike_ind] = pacbed.copy()

    return data


device = "cpu"
#device = "gpu"

In [ ]:
# select the file you want to load
file_adr = tkf.askopenfilename()
print(file_adr)

In [ ]:
# load a raw file and specify the calibration info
"""
dataset = py4DSTEM.io.import_file(file_adr)

Rx, Ry = 0.30038461538461536, 0.30038461538461536
R_unit = "A"
Qx, Qy = 1.07, 1.07
Q_unit = "mrad"

dataset.calibration._params['Q_pixel_size'] = Qx
dataset.calibration._params['Q_pixel_units'] = Q_unit
dataset.calibration._params['R_pixel_size'] = Rx
dataset.calibration._params['R_pixel_units'] = R_unit

print(dataset)
print(dataset.calibration)

HT = 200E3 # [V]
"""

In [ ]:
# load a tif file and specify the calibration info
_data = tifffile.imread(file_adr)
print(_data.shape)

dataset = py4DSTEM.DataCube(data=_data)

Rx, Ry = 0.3, 0.3
R_unit = "A"
Qx, Qy = 1.0, 1.0
Q_unit = "mrad"

dataset.calibration._params['Q_pixel_size'] = Qx
dataset.calibration._params['Q_pixel_units'] = Q_unit
dataset.calibration._params['R_pixel_size'] = Rx
dataset.calibration._params['R_pixel_units'] = R_unit

print(dataset)
print(dataset.calibration)

HT = 200E3 # [V]

In [ ]:
# (optional) invert the dimensions (a, b, c, d) -> (c, d, a, b)

dataset.data = fourd_roll_axis(dataset.data)

print(dataset)
print(dataset.calibration)

In [ ]:
# (optional)
dataset.data = np.nan_to_num(dataset.data) # NaN -> 0
#dataset.data = spike_remove(dataset.data, percent_thresh=0.01, mode="lower") # remove spike pixels (replace the spike pixels with the pacbed) -> optional stopgap

In [ ]:
dataset.get_dp_mean()
dataset.get_dp_max()

py4DSTEM.show(
    dataset.tree('dp_mean'),
    scaling = 'log',
    cmap = 'jet',
)

dataset.get_dp_mean()
dataset.get_dp_max()

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(dataset.tree('dp_mean')[:, :], cmap='jet')
fig.tight_layout()
plt.show()

probe_radius_pixels, probe_qx0, probe_qy0 = dataset.get_probe_size(thresh_lower=0.01, thresh_upper=0.99, N=100, plot=True)
plt.show()

In [ ]:
# Make a virtual bright field and dark field image
expand_BF = 2.0  # expand radius by 2 pixels to encompass the full center disk

center = (probe_qx0, probe_qy0)
radius_BF = probe_radius_pixels + expand_BF
radii_DF = (probe_radius_pixels + expand_BF, 1e3)

# Make a virtual bright field and dark field image
center = (probe_qx0, probe_qy0)
radius_BF = probe_radius_pixels
radii_DF = (probe_radius_pixels, int(dataset.Q_Nx/2))

dataset.get_virtual_image(
    mode = 'circle',
    geometry = (center,radius_BF),
    name = 'bright_field',
    shift_center = False,
)
dataset.get_virtual_image(
    mode = 'annulus',
    geometry = (center,radii_DF),
    name = 'dark_field',
    shift_center = False,
)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(dataset.tree('bright_field')[:, :], cmap="inferno")
ax[0].set_title("BF image")
ax[1].imshow(dataset.tree('dark_field')[:, :], cmap="inferno")
ax[1].set_title("ADF image [%.1f, %.1f] mrad"%(radii_DF[0]*dataset.Q_pixel_size, radii_DF[1]*dataset.Q_pixel_size))
fig.tight_layout()
plt.show()

In [ ]:
dpc = py4DSTEM.process.phase.DPC(
    datacube=dataset,
    energy = HT,
).preprocess()
plt.show()

In [ ]:
dpc.reconstruct(
    max_iter=8,
    store_iterations=True,
    reset=True,
    gaussian_filter_sigma=0.1,
    gaussian_filter=True,
    q_lowpass=None,
    q_highpass=None
).visualize(
    iterations_grid='auto',
    figsize=(16, 10)
)
plt.show()

In [ ]:
dpc_cor = py4DSTEM.process.phase.DPC(
    datacube=dataset,
    energy=HT,
    verbose=False,
).preprocess(
    force_com_rotation = np.rad2deg(dpc._rotation_best_rad),
    force_com_transpose = False,
)
plt.show()

In [ ]:
dpc_cor.reconstruct(
    max_iter=8,
    store_iterations=True,
    reset=True,
    gaussian_filter_sigma=0.1,
    gaussian_filter=True,
    q_lowpass=None,
    q_highpass=None
).visualize(
    iterations_grid='auto',
    figsize=(16, 10)
)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(dpc._com_normalized_y, cmap="bwr")
ax[0].set_title("CoMx")
ax[1].imshow(dpc._com_normalized_x, cmap="bwr")
ax[1].set_title("CoMy")
ax[2].imshow(np.sqrt(dpc._com_normalized_y**2 + dpc._com_normalized_x**2), cmap="inferno")
ax[2].set_title("Magnitude of CoM")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(dpc_cor._com_normalized_y, cmap="bwr")
ax[0].set_title("CoMx - rotation corrected")
ax[1].imshow(dpc_cor._com_normalized_x, cmap="bwr")
ax[1].set_title("CoMy - rotation corrected")
ax[2].imshow(np.sqrt(dpc_cor._com_normalized_y**2 + dpc_cor._com_normalized_x**2), cmap="inferno")
ax[2].set_title("Magnitude of CoM - rotation corrected")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(dpc.object_phase, cmap="inferno")
ax[0].set_title("iCoM")
ax[1].imshow(dpc_cor.object_phase, cmap="inferno")
ax[1].set_title("iCoM - rotation corrected")
fig.tight_layout()
plt.show()

In [ ]:
parallax = py4DSTEM.process.phase.Parallax(
    datacube=dataset,
    energy = HT,
    device = device, 
    verbose = True
).preprocess(
    normalize_images=True,
    plot_average_bf=False,
    edge_blend=8,
)

In [ ]:
parallax = parallax.reconstruct(
    reset=True,
    regularizer_matrix_size=(1,1),
    regularize_shifts=True,
    running_average=True,
    min_alignment_bin = 2,
    num_iter_at_min_bin = 4,
)
plt.show()

In [ ]:
parallax.show_shifts()
plt.show()

parallax.subpixel_alignment(
    #kde_upsample_factor=2,
    kde_sigma_px=0.125,
    plot_upsampled_BF_comparison=True,
    plot_upsampled_FFT_comparison=True,
)
plt.show()

In [ ]:
parallax.aberration_fit(
    plot_CTF_comparison=True,
)
plt.show()

parallax.aberration_correct(figsize=(5, 5))
plt.show()

In [ ]:
depth_sections = parallax.depth_section(depth_angstroms=np.arange(0, 200, 20), figsize=(12, 10))
plt.show()

In [ ]:
# Get the probe convergence semiangle from the pixel size and estimated radius in pixels
semiangle_cutoff = dataset.calibration.get_Q_pixel_size() * probe_radius_pixels
print('semiangle cutoff estimate = ' + str(np.round(semiangle_cutoff, decimals=1)) + ' mrads')

# Get the estimated defocus from the parallax reconstruction - note that defocus dF has the opposite sign as the C1 aberration!
defocus = -parallax.aberration_C1
print('estimated defocus         = ' + str(np.round(defocus)) + ' Angstroms')

rotation_degrees = np.rad2deg(parallax.rotation_Q_to_R_rads)
print('estimated rotation        = ' + str(np.round(rotation_degrees)) + ' deg')

In [ ]:
ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset,
    device = device,
    energy = HT,
    semiangle_cutoff = semiangle_cutoff,
    defocus = defocus,
    object_type='potential',
).preprocess(
    plot_center_of_mass = False, 
    plot_rotation = False, 
    plot_probe_overlaps = True, 
    force_com_rotation = rotation_degrees, 
    force_com_transpose = False,
    fit_function = "constant",
)

In [ ]:
ptycho_recon = ptycho.reconstruct(
    reset = True,
    store_iterations = True,
    max_iter = 20,
    #step_size = 0.5,
    #gaussian_filter_sigma = 0.3,
    normalization_min=1,
).visualize(
    iterations_grid= 'auto',
    figsize= (16,8),
)

In [ ]:
ptycho_recon.visualize(figsize=(16, 8))

In [ ]:
rotated_crop = ptycho_recon._crop_rotate_object_fov(ptycho.object)
rotated_shape = rotated_crop.shape

rotate_extent = [
    0,
    ptycho_recon.sampling[1] * rotated_shape[1],
    ptycho_recon.sampling[0] * rotated_shape[0],
    0,
]


fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(np.abs(ptycho_recon.object), cmap='inferno')
ax[1].imshow(np.abs(rotated_crop), cmap='inferno', extent=rotate_extent)
fig.tight_layout()
plt.show()

In [ ]:
n_probe = 5

ptycho_mix = py4DSTEM.process.phase.MixedstatePtychography(
    datacube=dataset,
    verbose=True,
    energy=HT,
    num_probes=n_probe,
    semiangle_cutoff=semiangle_cutoff,
    defocus=defocus,
    device=device,
    object_type='potential',
).preprocess(
    plot_center_of_mass = False,
    plot_rotation = False,
)

In [ ]:
ptycho_recon = ptycho_mix.reconstruct(
    reset=True,
    store_iterations=True,
    max_iter = 20,
    normalization_min= 1,
    #gaussian_filter_sigma=0.02,
    #step_size=0.5,
).visualize(
    iterations_grid= 'auto',
    figsize= (16,8)
)

In [ ]:
ptycho_recon.visualize(figsize=(16, 8))

In [ ]:
rotated_crop = ptycho_recon._crop_rotate_object_fov(ptycho_recon.object)
rotated_shape = rotated_crop.shape

rotate_extent = [
    0,
    ptycho_recon.sampling[1] * rotated_shape[1],
    ptycho_recon.sampling[0] * rotated_shape[0],
    0,
]


fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(np.abs(ptycho_recon.object), cmap='inferno')
ax[1].imshow(np.abs(rotated_crop), cmap='inferno', extent=rotate_extent)
fig.tight_layout()
plt.show()

In [ ]:
intensity_arrays = np.array([np.abs(probe)**2 for probe in ptycho_recon.probe])
probe_ratio      = [np.sum(intensity_array)/intensity_arrays.sum() for intensity_array in intensity_arrays]

py4DSTEM.show(
    [
        np.abs(ptycho_recon.probe_fourier[0])**2,
        np.abs(ptycho_recon.probe_fourier[1])**2
    ],
    scalebar=True,
    pixelsize=ptycho_recon.angular_sampling[0],
    pixelunits="mrad",
    ticks=False,
    title=[
        f"Probe 0 intensity: {probe_ratio[0]*100:.1f}%",
        f"Probe 1 intensity: {probe_ratio[1]*100:.1f}%"
    ]
)

In [ ]:
num_slice = 8
slice_thickness = 2.23

ms_ptycho = py4DSTEM.process.phase.MultislicePtychography(
    datacube=dataset,
    num_slices=num_slice,
    slice_thicknesses=slice_thickness,
    verbose=True,
    energy=HT,
    defocus=defocus,
    semiangle_cutoff=semiangle_cutoff,
    device=device,
).preprocess(
    plot_center_of_mass = False,
    plot_rotation=False,
)

In [ ]:
ptycho_recon = ms_ptycho.reconstruct(
    reset=True,
    store_iterations=True,
    max_iter = 20,
    normalization_min=1,
).visualize(
    iterations_grid = 'auto'
)

In [ ]:
ptycho_recon.visualize(figsize=(16, 8))

In [ ]:
fig, ax = plt.subplots(1, num_slice+1, figsize=(4*(num_slice+1), 4))

slice_object = ptycho_recon.object_iterations[29]

for i in range(num_slice):
    rotated_crop = ptycho_recon._crop_rotate_object_fov(slice_object[i])
    rotated_shape = rotated_crop.shape

    rotate_extent = [
        0,
        ptycho_recon.sampling[1] * rotated_shape[1],
        ptycho_recon.sampling[0] * rotated_shape[0],
        0,
    ]
    ax[i].imshow(np.angle(rotated_crop), cmap='inferno', extent=rotate_extent)

sum_object = ptycho_recon._crop_rotate_object_fov(np.sum(slice_object, axis=0))
ax[-1].imshow(np.angle(sum_object), cmap='inferno', extent=rotate_extent)
fig.tight_layout()
plt.show()

In [ ]:
ms_ptycho = py4DSTEM.process.phase.MultislicePtychography(
    datacube=dataset,
    num_slices=num_slice,
    slice_thicknesses=slice_thickness,
    verbose=True,
    energy=HT,
    defocus=defocus,
    semiangle_cutoff=semiangle_cutoff,
    device=device,
).preprocess(
    plot_center_of_mass = False,
    plot_rotation=False,
)

In [ ]:
ms_ptycho_tune = ms_ptycho.tune_num_slices_and_thicknesses(
        num_slices_guess=num_slice,
        thicknesses_guess=slice_thickness,
        num_slices_step_size=1,
        thicknesses_step_size=20,
        num_slices_values=3,
        num_thicknesses_values=3,
        update_defocus=False,
        max_iter=5,
        plot_reconstructions=True,
        plot_convergence=True,
        return_values=True,
)

In [ ]:
num_slice = 8
slice_thickness = 2.23
n_probe = 5

mms_ptycho = py4DSTEM.process.phase.MixedstateMultislicePtychography(
    datacube=dataset,
    num_slices=num_slice,
    slice_thicknesses=slice_thickness,
    num_probes=n_probe,
    verbose=True,
    energy=HT,
    defocus=defocus,
    semiangle_cutoff=semiangle_cutoff,
    device=device,
).preprocess(
    plot_center_of_mass = False,
    plot_rotation=False,
)

In [ ]:
ptycho_recon = mms_ptycho.reconstruct(
    reset=True,
    store_iterations=True,
    max_iter = 20,
    normalization_min=1,
).visualize(
    iterations_grid = 'auto'
)

In [ ]:
ptycho_recon.visualize(figsize=(16, 8))

In [ ]:
fig, ax = plt.subplots(1, num_slice+1, figsize=(4*(num_slice+1), 4))

slice_object = ptycho_recon.object_iterations[29]

for i in range(num_slice):
    rotated_crop = ptycho_recon._crop_rotate_object_fov(slice_object[i])
    rotated_shape = rotated_crop.shape

    rotate_extent = [
        0,
        ptycho_recon.sampling[1] * rotated_shape[1],
        ptycho_recon.sampling[0] * rotated_shape[0],
        0,
    ]
    ax[i].imshow(np.angle(rotated_crop), cmap='inferno', extent=rotate_extent)

sum_object = ptycho_recon._crop_rotate_object_fov(np.sum(slice_object, axis=0))
ax[-1].imshow(np.angle(sum_object), cmap='inferno', extent=rotate_extent)
fig.tight_layout()
plt.show()